# Worksheet Solutions

## 2.2c

In [ ]:
cahpbData.convert_units('kg m-2 day-1')
cahpbData.units = 'mm day-1'
cahpbData.remove_coord('forecast_period')
cahpbData.remove_coord('forecast_reference_time')
iris.save(cahpbData, path_in_cahpb + 'cahpb.pm.1981_1983.pr.norim.mmday-1.nc')

## 2.3e

In [ ]:
infile = os.path.join(DATADIR, 'APHRODITE', 'aphro.mon.6190.nc')
aphro_cube = iris.load_cube(infile)
iris.coord_categorisation.add_season(aphro_cube, 'time', name='seasons', seasons=('jfmamjjas','ond'))
aphro_cube_ond = aphro_cube.extract(iris.Constraint(seasons='ond'))
seasonal_aphro_mean = aphro_cube_ond.aggregated_by(['seasons'], iris.analysis.MEAN)
aphro_outfile = os.path.join(outdir, 'aphro.OND.mean.baseline.pr.mmday-1.nc')
iris.save(seasonal_aphro_mean, aphro_outfile)
print('APHRODITE: \n{}'.format(seasonal_aphro_mean))

# 3.1a

In [ ]:
# Solution: 
lon=(101.25, 102.15)
lat=(2.74, 3.48)